In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
import sympy as sy
import scipy.constants as pc
from sympy import *

In [3]:
# Funktionen
def am(werte):
    return np.mean(werte)

def std(werte):
    summen_der_quadrate = 0
    for wert in werte:
        summen_der_quadrate+=(wert-am(werte))**2       
    return np.sqrt(summen_der_quadrate/(werte.size-1))

def stu(werte):
    return std(werte)/np.sqrt(werte.size)

def fehler(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += (dif * unsicherheiten[i])**2
    zMax = sqrt(zMax)
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def fehlerMax(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += dif * unsicherheiten[i]
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def groessterBeitrag(f, werte, unsicherheiten, variablen):
    zMax = 0
    zMaxArray = np.array([])
    for i in range (werte.size):
        zMax = (np.abs(sy.diff(f, variablen[i])) * unsicherheiten[i])
        for j in range(werte.size):
            zMax = sub(zMax, variablen[j], werte[j])
        x = np.array([zMax])
        zMaxArray = np.append(zMaxArray, x)
    return variablen[np.argmax(zMaxArray)]

def sub(zMax, variable, wert):
    return zMax.subs([(variable, wert)])

def einfachGaussAdd(unsicherheiten, koeffizienten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (unsicherheiten[i]*unsicherheiten[i])**2
    return sqrt(dz)

def einfachGaussMult(unsicherheiten, werte, exponenten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (exponenten[i]*unsicherheiten[i]/werte[i])**2
    return sqrt(dz)

def vertraeglichkeit(am_1, am_2, stu_1, stu_2):
    return (np.abs(am_1-am_2)/(sqrt((stu_1**2)*(stu_2**2))))

def bestA(x, y):
    return (sum(x**2)*sum(y)-sum(x)*sum(x*y))/(x.size*sum(x**2)-sum(x)**2)

def bestB(x, y):
    return (x.size*sum(x*y)-sum(x)*sum(y))/(x.size*sum(x**2)-sum(x)**2)

def streuung(x, y):
    s = 0
    for i in range(x.size):
        s += (y[i]+(bestA(x, y)+bestB(x, y)*x[i]))**2
    return np.sqrt(s/(x.size-2))

def stuA(x, y):
    s = streuung(x, y)
    return s*np.sqrt(sum(x**2)/((x.size*sum(x**2)-sum(x)**2)))

def stuB(x, y):
    s = streuung(x, y)
    return s*np.sqrt(x.size/((x.size*sum(x**2)-sum(x)**2)))

In [13]:
# Objektdaten in mm
l_leg = 33.50 # +-0.05
b_leg = 12.57 # +- 0.01
h_leg = (15+6.45)/2 # +- 0.01

werte_leg = np.array([l_leg, b_leg, h_leg])
us_leg = np.array([0.05, 0.01, 0.01])
exp_leg = np.array([1, 1, 1])

vol_leg = l_leg*b_leg*h_leg
vol_leg_f = einfachGaussMult(us_leg, werte_leg, exp_leg)

vol_leg = 8240.47
vol_leg_f = 136.10

print('vol_leg: ', vol_leg)
print('vol_leg_f: ', vol_leg_f)


exp_zyl = np.array([1, 2])
werte_zyl = np.array([l_zyl, r_zyl])
us_zyl = np.array([0.05, 0.01])

vol_zyl = l_zyl*pc.pi*r_zyl**2
vol_zyl_f = einfachGaussMult(us_zyl, werte_zyl, exp_zyl)

print('vol_zyl: ', vol_zyl)
print('vol_zyl_f: ', vol_zyl_f)

# Kalibrierungsdaten
m_cal = np.array([0, 2, 5, 7, 10, 12, 15, 17, 20, 22, 25, 27, 30, 32, 35, 37, 40, 42, 45]) # g +-0.01g
us_m_cal = 0.01 #g
dx_cal = np.array([242, 231, 215, 205, 190, 179, 164, 152, 134, 124, 109, 98, 82, 71, 54, 44, 28, 18, 0]) # mm +- 1
us_dx_cal = 1.0

# Objektdaten
# 37, 18.5
# Cu Zn
dx_ob = np.array([44, 140])
us_dx_ob = 1


vol_leg:  8240.47
vol_leg_f:  136.1
vol_zyl:  2650.7668678339883
vol_zyl_f:  0.00510053592845024


In [14]:
a = 0.12328689876772127
delta_a = 0.06934679146053928

b = 0.18629652299531144
delta_b = 0.0004962585742822325

x_zyl = (140-242)*(-1)
x_leg = (44-242)*(-1)
delta_x = 2

#Fehler Masse Zylinder
#m = a + bx , m'(a)=1 , m'(x)=b, m'(b)=x

def delta_m(x):
    return np.sqrt((1 * delta_a)**2+(b*delta_x)**2+(x*delta_b)**2)

print("Unsicherheit Zylinder", delta_m(x_zyl))
print("Unischerheit Legierung", delta_m(x_leg))









Unsicherheit Zylinder 0.3823568689735257
Unischerheit Legierung 0.3915219346864727


In [18]:
dichte_zink = 0.00714 #g/mm^3 
dichte_zink_f = 0.00001

dichte_kupfer = 0.00896 #g/mm^3
dichte_kupfer_f = 0.00001

mas_leg = ((44-242)*-0.1862) + 0.12
mas_leg_f = delta_m(x_leg)
print(mas_leg)
print(mas_leg_f)

print(mas_leg/vol_leg,"dich")


#vol_leg 
#vol_leg_f
# mas_leg_f 2mm für Länge 
# 0.12 +- 0.06934679146053928
# 0.1862 +- 0.0004962585742822325
# mas_leg/vol_leg = dichte_leg

# dichte_leg = x*dichte_zink + y*dichte_kupfer

#mas_leg / vol_leg = anteil_zink * dichte_zink + anteil_kupfer * dichte_kupfer
#anteil_kupfer + anteil_zink = 1

#Umformen

anteil_kupfer = (mas_leg/vol_leg - dichte_zink)/(dichte_kupfer-dichte_zink)
print("Anteil Kupfer", 100 * anteil_kupfer,"%")

anteil_zink = 1 - anteil_kupfer
print("Anteil Zink",  100*anteil_zink,"%")

#mas_leg / vol_leg = anteil_zink * dichte_zink + anteil_kupfer * dichte_kupfer


36.9876
0.3915219346864727
0.004488530387223059 dich
Anteil Kupfer -145.6851435591726 %
Anteil Zink 245.68514355917256 %


In [49]:
#Fehler Anteil Kupfer/Zink

dichte_leg = 8240.47
dichte_leg_f = 136.10


#K = (P - dz)/(dk-dz)
#K'P = 1/(dk-dz) ; K'dz = -(dk-P)/(dz-dk)^2 ; K'dk = (dz-P)/(dk-dz)**2

#def fehler_anteile(P, dk, dz,  P_f, dk_f, dz_f):
#    print(P, P_f, dk, dk_f, dz, dz_f)
#    return np.sqrt((1/(dk-dz) * P_f)**2 + (-dz_f * (dk-P)/(dz-dk)**2)**2 + (dk * (dz-P)/(dk-dz)**2)**2)
    
#anteile_f = fehler_anteile(dichte_leg, dichte_kupfer, dichte_zink, dichte_leg_f, dichte_kupfer_f, dichte_kupfer_f)
#print(anteile_f)

unsich_oben = np.sqrt(dichte_leg_f**2 + dichte_zink_f**2)
unsich_unten = np.sqrt(dichte_zink_f**2 + dichte_kupfer_f**2)

expo_dich = np.array([1,-1])
unsicher = np.array([unsich_oben, unsich_unten])
wert = np.array([dichte_leg-dichte_zink, dichte_kupfer - dichte_zink])
print(einfachGaussMult(unsicher, wert, expo_dich)*100,"%")

1.82526574302663 %


1.0